## 爬取疫情数据并同步到sql

In [1]:
import urllib.request as rq
import pymysql
from bs4 import BeautifulSoup
import json
import time
import traceback

In [2]:
url_today = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
url_last = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
def gethtml(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
    req = rq.Request(url, headers=headers)
    res = rq.urlopen(req)

    html = res.read().decode("utf-8")
    return html


In [3]:
def  get_history(url_last, url_today):
    # 历史数据
    html_last = gethtml(url_last)
    data_last = json.loads(html_last)
    #dict_keys(['chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList', 'provinceCompare', 'cityStatis', 'nowConfirmStatis'])
    data_last = json.loads(data_last["data"])

    # 当天数据
    html_today = gethtml(url_today)
    data_today = json.loads(html_today)
    # dict_keys(['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree'])
    data_today = json.loads(data_today["data"])
    # print(data_today.keys())

    history_data = {}
    for every_data in data_last["chinaDayList"]:
        date = "2021." + every_data["date"]
        tup = time.strptime(date, "%Y.%m.%d") #匹配时间
        date = time.strftime("%Y-%m-%d", tup)  # 改变时间格式,不然插入数据库会报错，数据库是datetime类型

        confirm = every_data["confirm"]
        suspect = every_data["suspect"]
        dead = every_data["dead"]
        heal = every_data["heal"]
        # nowConfirm = every_data["nowConfirm"]

        history_data[date] = {"confirm":confirm, "suspect":suspect, "dead":dead, "heal":heal}

    for every_data_add in data_last["chinaDayAddList"]:
        date_add = "2021." + every_data_add["date"]
        tup_add = time.strptime(date_add, "%Y.%m.%d")  #匹配时间
        date_add = time.strftime("%Y-%m-%d", tup_add)  # 改变时间格式,不然插入数据库会报错，数据库是datetime类型

        confirm_add = every_data_add["confirm"]
        suspect_add = every_data_add["suspect"]
        dead_add = every_data_add["dead"]
        heal_add = every_data_add["heal"]
        # g更新，update函数可以添加新的键值对
        history_data[date_add].update({"confirm_add":confirm_add, "suspect_add":suspect_add, "dead_add":dead_add, "heal_add":heal_add})

    ''' areaTree ：name 中国数据
                   today
                   total
                   children :-name 省级数据 
                            -today
                            -total
                            -children:-name 市级数据
                                      -today
                                      -total
                            '''
    details = []  #当日详细数据
    update_time = data_today["lastUpdateTime"]
    data_country = data_today["areaTree"]  #获取到中国国家
    # print(data_country[0]["children"])
    data_province = data_country[0]["children"]   #中国省份 
    # print(data_province)
    for pro_infos in data_province:
        province = pro_infos["name"]   #省名  
        for city_infos in pro_infos["children"]:  #获取市的名称
            city = city_infos["name"]
            confirm = city_infos["total"]["confirm"]
            confirm_add = city_infos["today"]["confirm"]
            heal = city_infos["total"]["heal"]
            dead = city_infos["total"]["dead"]
            details.append([update_time, province, city, confirm, confirm_add, heal, dead])
    return history_data, details

In [4]:
#存储数据 其中history 表存储每日总数据，details表存储每日详细数据
# 安装pip install pymysql
#(1)建立连接
#（2）建立游标
#（3）执行操作
#（4）关闭连接

In [5]:
# 连接数据库
def get_conn():   #封装连接
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="123456",
        db="cov1",
        charset="utf8",
        port=3306,
    )
    # 创建游标：
    cursor = conn.cursor()  #执行完毕返回的结果集默认以元组显示
    return conn, cursor 
#（2）关闭连接
def close_conn(conn, cursor):   #执行完毕关闭连接和游标
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [6]:
#（3）定义更新细节数据函数
def update_details(url_last, url_today):
    cursor = None
    conn = None
    try:
        # [['2020-07-01 15:20:46', '北京', '丰台', 266, 0, 43, 0],....]
        detail_data = get_history(url_last, url_today)[1] # 0是字典数据 1是列表数据
        conn, cursor = get_conn()  #创立连接，连接游标
        #插入数据的sql语句，前面一开始的值，后面百分号做占位
        sql = "insert into details(update_time,province,city,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s,%s)"
        #该sql获的是数据库的最新时间
        sql_query = 'select %s=(select update_time from details order by id desc limit 1)' #对比当前最大时间戳,拿到最后一条数据
        #对比当前最大时间戳
        cursor.execute(sql_query, detail_data[0][0]) #这边的是从腾讯爬取出来的最新时间li[0]表示获取历史数据，li[0][0]获取的是历史数据的时间
        if not cursor.fetchone()[0]:  #上述两个时间不同
            print(f"{time.asctime()}开始更新最新数据")
            for item in detail_data:
                cursor.execute(sql, item)
            conn.commit()  # 提交事务 update delete insert操作
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据！")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)
#插入历史数据
def insert_history(url_last, url_today):
    cursor = None
    conn = None
    try:
        dic = get_history(url_last, url_today)[0] #0代表历史数据字典

        print(f"{time.asctime()}开始插入历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        for key, value in dic.items():
             # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            cursor.execute(sql, [key, value.get("confirm"), value.get("confirm_add"), value.get("suspect"),
                                 value.get("suspect_add"), value.get("heal"), value.get("heal_add"),
                                 value.get("dead"), value.get("dead_add")])
        conn.commit()  # 提交事务 update delete insert操作
        print(f"{time.asctime()}插入历史数据完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

#更新历史数据        
def update_history(url_last, url_today):
    cursor = None
    conn = None
    try:
        dic = get_history(url_last, url_today)[0]  #0代表历史数据字典
        print(f"{time.asctime()}开始更新历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        sql_query = "select confirm from history where ds=%s"
        for key, value in dic.items():
            # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            if not cursor.execute(sql_query, key):
                cursor.execute(sql, [key, value.get("confirm"), value.get("confirm_add"), value.get("suspect"),
                                     value.get("suspect_add"), value.get("heal"), value.get("heal_add"),
                                     value.get("dead"), value.get("dead_add")])
        conn.commit()
        print(f"{time.asctime()}历史数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [7]:
update_details(url_last, url_today)  #更新历史数据
update_history(url_last, url_today)  #插入历史数据

Tue Nov  2 20:30:38 2021开始更新最新数据
Tue Nov  2 20:30:39 2021更新最新数据完毕
Tue Nov  2 20:30:39 2021开始更新历史数据
Tue Nov  2 20:30:41 2021历史数据更新完毕


## 爬取百度热搜并保存到sql里面

In [8]:
from selenium import webdriver
import time
import sys
import pymysql
import json
import traceback
import requests

In [9]:
#写入数据库
#爬取百度热搜数据
def get_baidu_hot():
    url = "https://top.baidu.com/board?tab=realtime"
    path="D:\Data\jupyterNote Data\chromedriver.exe"  #可以指定驱动的位置，也可以放在.py同级目录下就不用设置path了
    option=webdriver.ChromeOptions()
    option.add_argument('--headless') #隐藏浏览器
    option.add_argument("--no-sandbox")
    browser=webdriver.Chrome(path,options=option)
    

    
    browser.get(url)  # 2.打开地址
    time.sleep(1)
    # 爬虫与反爬，模拟人等待1秒
    c = browser.find_elements_by_xpath('//*[@id="sanRoot"]/main/div[2]/div/div[2]/div/div[2]/a/div[1]')  #共有30条记录
    context = [i.text for i in c]  #i.text获取标签内容，插入到list中来
    print(context)
    browser.close()
    return context


def update_hotsearch():
    cursor = None
    conn = None
    try:
        context = get_baidu_hot()  #得到内容
        print(f"{time.asctime()}开始更新数据")
        conn,cursor = get_conn() #拿取到了连接
        sql = "insert into hotsearch(dt,content) values(%s,%s)" 
        ts = time.strftime("%Y-%m-%d %X")
        for i in context:
            cursor.execute(sql,(ts,i))  #插入数据
        conn.commit()  #提交事务保存数据
        print(f"{time.asctime()}数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn,cursor)

In [10]:
update_hotsearch()

['鼓励储存必需品引热议 商务部回应', '全能型冷空气将横扫 南北都要冻哭', '美核潜艇撞上海底山?外交部回应', '上海迪士尼11月3日起恢复运营', '渤海银行涉事员工目前已失联', '成都新增确诊与兰州病例有轨迹重叠', '马斯克发中文七步诗', '600亿基金经理蔡向阳身故 年仅41岁', '重庆九龙坡区新增1例确诊', '莫迪承诺印度2070年将实现净零排放', '济南市民收到战备应急包 官方回应', '宁夏吴忠回应男子发狗狗表情包被拘', '北京就限票政策“误伤”道歉', '河北石家庄进入应急状态', '江苏回应发布家庭应急物资清单', '钟南山:中国防疫政策并非成本过高', '雅虎在中国大陆停止产品与服务', '北京一家4口确诊 1人每日接送儿子', '拜登替特朗普政府向全世界道歉', '叶问3制片人施建祥在美被捕', '#李诚儒痛批娱乐圈娘炮文化#', '#范丞丞空降聊致命愿望#', '平顶山回应公交司机集体停运讨薪', '神十三航天员起床洗漱画面曝光', '事业单位招聘不得限制毕业院校', '迪士尼归来游客称自费隔离要6千', '耿爽用中国俗话正告个别国家', '重返英超!孔蒂执教热刺', '16省538例 百人规模疫情为何频发', '高校食堂男生因抢座狂扇女生耳光']
Tue Nov  2 20:30:47 2021开始更新数据
Tue Nov  2 20:30:49 2021数据更新完毕
